## Imports

In [2]:
import os

## Deleteing unneeded folders

In [23]:
mainFolderLocation = "ABI-1 Corpus/accents/"
listOfAccents = os.listdir(mainFolderLocation)

# Goes through list of accent folders
for accent in listOfAccents:
    accentFolderLocation = mainFolderLocation + accent + "/"
    if accent == ".DS_Store":
        os.remove(accentFolderLocation)

    accentContent = os.listdir(accentFolderLocation)

    # Goes through list of gender folders
    for genders in accentContent:
        genderFolderLocation = accentFolderLocation + genders + "/"
        if genders == ".DS_Store":
          os.remove(genderFolderLocation)

        genderContent = os.listdir(genderFolderLocation)
        
        # Goes through list of participant folders
        for participant in genderContent:
            participantFolderLocation = genderFolderLocation + participant + "/"
            if participant == ".DS_Store":
                os.remove(participantFolderLocation)

            participantContent = os.listdir(participantFolderLocation)

            for file in participantContent:
                fileAddress = participantFolderLocation + file
                if file[-3:] != "wav":
                    os.remove(fileAddress)

                if file[:12] != "shortpassage":
                    os.remove(fileAddress)


## Obtaining List of directories

In [57]:
def getSpeakerRoots(datapath = "ABI-1 Corpus/accents"):
    speakerList = []
    accentSubfolder = [f.path for f in os.scandir(datapath) if f.is_dir()]
    for accent in accentSubfolder:
        for gender in ["female", "male"]:
            speakerFolders = os.listdir(os.path.join(accent, gender))
            for speaker in speakerFolders:
                if not speaker.startswith("."):
                    speakerList.append(os.path.join(accent, gender, speaker))
        
    return speakerList

In [64]:
speakerList = getSpeakerRoots()
for i in speakerList:
  print(i)

ABI-1 Corpus/accents\brm_001\female\alw001
ABI-1 Corpus/accents\brm_001\female\cxb001
ABI-1 Corpus/accents\brm_001\female\jah001
ABI-1 Corpus/accents\brm_001\female\jep001
ABI-1 Corpus/accents\brm_001\female\knb001
ABI-1 Corpus/accents\brm_001\female\lcg001
ABI-1 Corpus/accents\brm_001\female\lst001
ABI-1 Corpus/accents\brm_001\female\mjd001
ABI-1 Corpus/accents\brm_001\female\mpt001
ABI-1 Corpus/accents\brm_001\female\rkk001
ABI-1 Corpus/accents\brm_001\male\ajh001
ABI-1 Corpus/accents\brm_001\male\apd001
ABI-1 Corpus/accents\brm_001\male\ars001
ABI-1 Corpus/accents\brm_001\male\axm001
ABI-1 Corpus/accents\brm_001\male\ikp001
ABI-1 Corpus/accents\brm_001\male\ixj001
ABI-1 Corpus/accents\brm_001\male\jxs001
ABI-1 Corpus/accents\brm_001\male\mam001
ABI-1 Corpus/accents\brm_001\male\mrs001
ABI-1 Corpus/accents\brm_001\male\wxh001
ABI-1 Corpus/accents\crn_001\female\cxc001
ABI-1 Corpus/accents\crn_001\female\dmc001
ABI-1 Corpus/accents\crn_001\female\ejr001
ABI-1 Corpus/accents\crn_001\fe

## Getting .wav files

In [65]:
def getWavFile(datapath):
    files = os.listdir(datapath)
    filesWav = [i for i in files if i.endswith(".wav")]
    return filesWav

In [99]:
speakers = []
for i in speakerList:
    speakers.append(i[-6:])

speakerFilePairs = {}
for i in range(len(speakers)):
    speakerFilePairs[speakers[i]] = getWavFile(speakerList[i])

for i in speakerFilePairs:
    print(i, speakerFilePairs[i])

alw001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
cxb001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
jah001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
jep001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
knb001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
lcg001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
lst001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
mjd001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
mpt001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
rkk001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
ajh001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
apd001 ['shortpassagea_CT.wav', 'shortpassageb_CT.wav', 'shortpassagec_CT.wav']
ars001 ['shortpassagea_CT.wav', 'shortpa